In [1]:
import nltk
nltk.download('brown')  # you only have to run this once

[nltk_data] Downloading package brown to /home/jovyan/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.


True

In [2]:
import sys
from nltk.corpus import brown
import pandas
from io import StringIO

In [3]:
# Hidden Markov Models in Python
# Katrin Erk, March 2013 updated March 2016
#
# This HMM addresses the problem of part-of-speech tagging. It estimates
# the probability of a tag sequence for a given word sequence as follows:
#
# Say words = w1....wN
# and tags = t1..tN
#
# then
# P(tags | words) is_proportional_to  product P(ti | t{i-1}) P(wi | ti)
#
# To find the best tag sequence for a given sequence of words,
# we want to find the tag sequence that has the maximum P(tags | words)

In [4]:
# Estimating P(wi | ti) from corpus data using Maximum Likelihood Estimation (MLE):
# P(wi | ti) = count(wi, ti) / count(ti)
#
# We add an artificial "start" tag at the beginning of each sentence, and
# We add an artificial "end" tag at the end of each sentence.
# So we start out with the brown tagged sentences,
# add the two artificial tags,
# and then make one long list of all the tag/word pairs.

brown_tags_words = [ ]
for sent in brown.tagged_sents():
    # sent is a list of word/tag pairs
    # add START/START at the beginning
    brown_tags_words.append( ("START", "START") )
    # then all the tag/word pairs for the word/tag pairs in the sentence.
    # shorten tags to 2 characters each
    brown_tags_words.extend([ (tag[:2], word) for (word, tag) in sent ])
    # then END/END
    brown_tags_words.append( ("END", "END") )

# conditional frequency distribution
cfd_tagwords = nltk.ConditionalFreqDist(brown_tags_words)
# conditional probability distribution
cpd_tagwords = nltk.ConditionalProbDist(cfd_tagwords, nltk.MLEProbDist)

print("The probability of an adjective (JJ) being 'new' is", cpd_tagwords["JJ"].prob("new"))
print("The probability of a verb (VB) being 'duck' is", cpd_tagwords["VB"].prob("duck"))
print("The probability of a verb (NN) being 'duck' is", cpd_tagwords["NN"].prob("duck"))

The probability of an adjective (JJ) being 'new' is 0.01472344917632025
The probability of a verb (VB) being 'duck' is 6.042713350943527e-05
The probability of a verb (NN) being 'duck' is 8.779669796618949e-06


In [5]:
# Estimating P(ti | t{i-1}) from corpus data using Maximum Likelihood Estimation (MLE):
# P(ti | t{i-1}) = count(t{i-1}, ti) / count(t{i-1})
brown_tags = [tag for (tag, word) in brown_tags_words ]
print(brown_tags[0:30])

# make conditional frequency distribution:
# count(t{i-1} ti)
cfd_tags= nltk.ConditionalFreqDist(nltk.bigrams(brown_tags))
# make conditional probability distribution, using
# maximum likelihood estimate:
# P(ti | t{i-1})
cpd_tags = nltk.ConditionalProbDist(cfd_tags, nltk.MLEProbDist)

print("If we have just seen 'DT', the probability of 'NN' is", cpd_tags["DT"].prob("NN"))
print("If we have just seen 'DT', the probability of 'JJ' is", cpd_tags["DT"].prob("JJ"))
print( "If we have just seen 'VB', the probability of 'JJ' is", cpd_tags["VB"].prob("JJ"))
print( "If we have just seen 'VB', the probability of 'NN' is", cpd_tags["VB"].prob("NN"))

['START', 'AT', 'NP', 'NN', 'JJ', 'NN', 'VB', 'NR', 'AT', 'NN', 'IN', 'NP', 'JJ', 'NN', 'NN', 'VB', '``', 'AT', 'NN', "''", 'CS', 'DT', 'NN', 'VB', 'NN', '.', 'END', 'START', 'AT', 'NN']
If we have just seen 'DT', the probability of 'NN' is 0.5057722522030194
If we have just seen 'DT', the probability of 'JJ' is 0.10506182116264773
If we have just seen 'VB', the probability of 'JJ' is 0.03443483365273389
If we have just seen 'VB', the probability of 'NN' is 0.10970977711020183


In [6]:
def tabulate_cfd(cfd, *args, **kwargs):
    """
    Tabulate the given samples from the conditional frequency distribution
    or conditional probability distribution.

    :param samples: The samples to plot
    :type samples: list
    :param conditions: The conditions to plot (default is all)
    :type conditions: list
    :param cumulative: A flag to specify whether the freqs are cumulative
    (default = False)
    :type title: bool
    """

    cumulative = False
    # conditions = sorted([c for c in cfd.conditions()
    #                     if "_t_" in c])   # only concerned with act-final
    conditions = sorted([c for c in cfd.conditions()])
    if type(cfd) == nltk.ConditionalProbDist:
        samples = sorted(set(v for c in conditions for v in cfd[c].samples()))
    else:
        samples = sorted(set(v for c in conditions for v in cfd[c]))
    if samples == []:
        print("No conditions for tabulate!")
        return None
    width = max(len("%s" % s) for s in samples)
    freqs = dict()
    for c in conditions:
        if cumulative:
            freqs[c] = list(cfd[c]._cumulative_frequencies(samples))
        else:
            if type(cfd) == nltk.ConditionalProbDist:
                freqs[c] = [cfd[c].prob(sample) for sample in samples]
            else:
                freqs[c] = [cfd[c][sample] for sample in samples]
        width = max(width, max(len("%d" % f) for f in freqs[c]))

    # condition_size = max(len("%s" % c) for c in conditions)
    final_string = ""
    # final_string += ' ' * condition_size
    if type(cfd) == nltk.ConditionalProbDist:
        width += 1
    for s in samples:
        # final_string += "%*s" % (width, s)
        final_string += "\t" + str(s)
    final_string = final_string + "\n"
    for c in conditions:
        # final_string += "%*s" % (condition_size, c)
        final_string += str(c) + "\t"
        for f in freqs[c]:

            if type(cfd) == nltk.ConditionalProbDist:
                # final_string += "%*.2f" % (width, f)
                if f == 0:
                    final_string += "\t"
                else:
                    final_string += "{}\t".format(f)
            else:
                # final_string += "%*d" % (width, f)
                if f == 0:
                    final_string += "\t"
                else:
                    final_string += "{}\t".format(f)
        final_string = final_string[:-1] + "\n"
    return final_string

In [7]:
# We can have a look at the transition probabilities in a matrix
df = pandas.read_csv(StringIO(tabulate_cfd(cpd_tags)), delimiter="\t")
df.fillna("-")  # GET RID OF NANS

,Unnamed: 0,','',(,(-,),)-,*,*-,",",...,RP,START,TO,UH,VB,WD,WP,WQ,WR,``
0,',-,0.126183,-,-,0.006309,-,-,0.003155,0.066246,...,-,-,0.003155,-,0.082019,0.006309,0.003155,-,0.015773,0.003155
1,'',-,-,0.006827,-,0.004779,-,0.000114,-,0.231198,...,0.000228,-,0.002958,-,0.017522,0.002844,0.002276,-,0.001365,0.000569
2,(,0.000442,0.000442,0.006184,-,0.000442,-,0.004417,-,0.000883,...,0.001325,-,0.002208,-,0.065813,0.015459,0.0053,0.001767,0.004417,0.016343
3,(-,-,-,-,-,-,-,-,0.006173,-,...,-,-,-,-,0.006173,-,-,-,-,-
4,),0.00044,0.004839,0.00044,-,0.00396,-,0.00088,-,0.172899,...,-,-,0.006599,0.00044,0.067312,0.006599,0.010119,-,0.006599,0.004839
5,)-,-,-,-,-,-,-,-,-,-,...,-,-,-,-,0.005435,-,-,-,-,-
6,*,0.000217,0.003041,-,-,0.000434,-,-,-,0.016511,...,0.001086,-,0.034325,0.000217,0.368238,0.002172,0.000434,0.000217,0.001303,0.000869
7,*-,-,-,-,-,-,-,-,-,-,...,-,-,-,-,0.3,-,-,-,-,-
8,",",0.000567,0.000189,0.00086,0.000069,0.000086,-,0.004471,-,0.001238,...,0.000911,-,0.007841,0.001496,0.107968,0.013395,0.011847,0.000464,0.011865,0.018794
9,",-",-,-,-,0.005556,-,-,-,-,0.005556,...,-,-,0.005556,-,0.016667,0.005556,-,-,-,-


In [8]:
###
# putting things together:
# what is the probability of the tag sequence "PP VB TO VB" for the word sequence "I want to race"?
# (though not normalized by p("I want to race"), as it's not needed for classification/tagging)
# It is
# P(START) * P(PP|START) * P(I | PP) *
#            P(VB | PP) * P(want | VB) *
#            P(TO | VB) * P(to | TO) *
#            P(VB | TO) * P(race | VB) *
#            P(END | VB)
#
# We leave aside P(START) for now.

prob_tagsequence = cpd_tags["START"].prob("PP") * cpd_tagwords["PP"].prob("I") * \
    cpd_tags["PP"].prob("VB") * cpd_tagwords["VB"].prob("want") * \
    cpd_tags["VB"].prob("TO") * cpd_tagwords["TO"].prob("to") * \
    cpd_tags["TO"].prob("VB") * cpd_tagwords["VB"].prob("race") * \
    cpd_tags["VB"].prob("END")

print( "The probability of the tag sequence 'START PP VB TO VB END' for 'I want to race' is:", prob_tagsequence)

prob_tagsequence = cpd_tags["START"].prob("PP") * cpd_tagwords["PP"].prob("I") * \
    cpd_tags["PP"].prob("VB") * cpd_tagwords["VB"].prob("saw") * \
    cpd_tags["VB"].prob("PP") * cpd_tagwords["PP"].prob("her") * \
    cpd_tags["PP"].prob("NN") * cpd_tagwords["NN"].prob("duck") * \
    cpd_tags["NN"].prob("END")

print( "The probability of the tag sequence 'START PP VB PP NN END' for 'I saw her duck' is:", prob_tagsequence)

prob_tagsequence = cpd_tags["START"].prob("PP") * cpd_tagwords["PP"].prob("I") * \
    cpd_tags["PP"].prob("VB") * cpd_tagwords["VB"].prob("saw") * \
    cpd_tags["VB"].prob("PP") * cpd_tagwords["PP"].prob("her") * \
    cpd_tags["PP"].prob("VB") * cpd_tagwords["VB"].prob("duck") * \
    cpd_tags["VB"].prob("END")

print( "The probability of the tag sequence 'START PP VB PP VB END' for 'I saw her duck' is:", prob_tagsequence)

The probability of the tag sequence 'START PP VB TO VB END' for 'I want to race' is: 1.0817766461150474e-14
The probability of the tag sequence 'START PP VB PP NN END' for 'I saw her duck' is: 3.372745049070759e-16
The probability of the tag sequence 'START PP VB PP VB END' for 'I saw her duck' is: 7.285965712199413e-16


In [9]:
#####
# Viterbi:
# If we have a word sequence, what is the best tag sequence?
#
# The method above lets us determine the probability for a single tag sequence.
# But in order to find the best tag sequence, we need the probability
# for _all_ tag sequence.
# What Viterbi gives us is just a good way of computing all those many probabilities
# as fast as possible.

# what is the list of all tags?
distinct_tags = set(brown_tags)
print(distinct_tags)

#try different sentences:
#sentence = ["I", "want", "to", "race" ]
#sentence = ["I", "saw", "her", "duck" ]
sentence = ["fruit", "flies", "are", "nice"]
#sentence = ["fruit", "flies", "like", "an", "arrow"]
sentlen = len(sentence)

# viterbi:
# for each step i in 1 .. sentlen,
# store a dictionary
# that maps each tag X
# to the probability of the best tag sequence of length i that ends in X
viterbi = [ ]

# backpointer:
# for each step i in 1..sentlen,
# store a dictionary
# that maps each tag X
# to the previous tag in the best tag sequence of length i that ends in X
backpointer = [ ]

# Initialize first column of cells by getting probs of tags given first word and start symbol
first_viterbi = { }
first_backpointer = { }
for tag in distinct_tags:
    # don't record anything for the START tag
    if tag == "START": continue
    first_viterbi[ tag ] = cpd_tags["START"].prob(tag) * cpd_tagwords[tag].prob( sentence[0] )
    first_backpointer[ tag ] = "START"

print("Initialized first cell:")
print(first_viterbi)
print(first_backpointer)
    
viterbi.append(first_viterbi)
backpointer.append(first_backpointer)

current_best = max(first_viterbi.keys(), key = lambda tag: first_viterbi[ tag ])
print( "Word", "'" + sentence[0] + "'", "current best two-tag sequence:", first_backpointer[ current_best], current_best)
# print( "Word", "'" + sentence[0] + "'", "current best tag:", current_best)

{'--', 'OD', 'FW', 'RN', 'HV', ',', ':-', 'NN', 'IN', 'END', 'CS', 'NI', 'PN', '*', "''", 'BE', '``', 'TO', 'RP', 'WD', 'MD', 'WQ', 'UH', 'WR', 'NP', '(-', 'START', 'AP', 'WP', 'DO', 'EX', 'AT', 'VB', ')', ':', 'PP', "'", 'NR', ',-', 'CD', 'DT', '(', '.-', 'RB', ')-', 'QL', 'JJ', '.', 'AB', '*-', 'CC'}
Initialized first cell:
{'--': 0.0, 'OD': 0.0, 'FW': 0.0, 'RN': 0.0, 'HV': 0.0, ',': 0.0, ':-': 0.0, 'NN': 8.464250895659148e-06, 'IN': 0.0, 'END': 0.0, 'CS': 0.0, 'NI': 0.0, 'PN': 0.0, '*': 0.0, "''": 0.0, 'BE': 0.0, '``': 0.0, 'TO': 0.0, 'RP': 0.0, 'WD': 0.0, 'MD': 0.0, 'WQ': 0.0, 'UH': 0.0, 'WR': 0.0, 'NP': 0.0, '(-': 0.0, 'AP': 0.0, 'WP': 0.0, 'DO': 0.0, 'EX': 0.0, 'AT': 0.0, 'VB': 0.0, ')': 0.0, ':': 0.0, 'PP': 0.0, "'": 0.0, 'NR': 0.0, ',-': 0.0, 'CD': 0.0, 'DT': 0.0, '(': 0.0, '.-': 0.0, 'RB': 0.0, ')-': 0.0, 'QL': 0.0, 'JJ': 0.0, '.': 0.0, 'AB': 0.0, '*-': 0.0, 'CC': 0.0}
{'--': 'START', 'OD': 'START', 'FW': 'START', 'RN': 'START', 'HV': 'START', ',': 'START', ':-': 'START', 'NN'

In [10]:
for wordindex in range(1, len(sentence)):
    this_viterbi = { }
    this_backpointer = { }
    prev_viterbi = viterbi[-1]
    
    for tag in distinct_tags:
        # don't record anything for the START tag
        if tag == "START": continue

        # if this tag is X and the current word is w, then 
        # find the previous tag Y such that
        # the best tag sequence that ends in X
        # actually ends in Y X
        # that is, the Y that maximizes
        # prev_viterbi[ Y ] * P(X | Y) * P( w | X)
        # The following command has the same notation
        # that you saw in the sorted() command.
        best_previous = max(prev_viterbi.keys(),
                            key = lambda prevtag: \
            prev_viterbi[ prevtag ] * cpd_tags[prevtag].prob(tag) * cpd_tagwords[tag].prob(sentence[wordindex]))

        # Instead, we can also use the following longer code:
        # best_previous = None
        # best_prob = 0.0
        # for prevtag in distinct_tags:
        #    prob = prev_viterbi[ prevtag ] * cpd_tags[prevtag].prob(tag) * cpd_tagwords[tag].prob(sentence[wordindex])
        #    if prob > best_prob:
        #        best_previous= prevtag
        #        best_prob = prob
        #
        this_viterbi[ tag ] = prev_viterbi[ best_previous] * \
            cpd_tags[ best_previous ].prob(tag) * cpd_tagwords[ tag].prob(sentence[wordindex])
        this_backpointer[ tag ] = best_previous

    current_best = max(this_viterbi.keys(), key = lambda tag: this_viterbi[ tag ])
    print( "Word", "'" + sentence[ wordindex] + "'", "current best two-tag sequence:", this_backpointer[ current_best], current_best)
    # print( "Word", "'" + sentence[ wordindex] + "'", "current best tag:", current_best)


    # done with all tags in this iteration
    # so store the current viterbi step
    viterbi.append(this_viterbi)
    backpointer.append(this_backpointer)


# done with all words in the sentence.
# now find the probability of each tag
# to have "END" as the next tag,
# and use that to find the overall best sequence
prev_viterbi = viterbi[-1]
best_previous = max(prev_viterbi.keys(),
                    key = lambda prevtag: prev_viterbi[ prevtag ] * cpd_tags[prevtag].prob("END"))

prob_tagsequence = prev_viterbi[ best_previous ] * cpd_tags[ best_previous].prob("END")

# best tagsequence: we store this in reverse for now, will invert later
best_tagsequence = [ "END", best_previous ]
# invert the list of backpointers
backpointer.reverse()

# go backwards through the list of backpointers
# (or in this case forward, because we have inverter the backpointer list)
# in each case:
# the following best tag is the one listed under
# the backpointer for the current best tag
current_best_tag = best_previous
for bp in backpointer:
    best_tagsequence.append(bp[current_best_tag])
    current_best_tag = bp[current_best_tag]

best_tagsequence.reverse()
print( "The sentence was:", end = " ")
for w in sentence: print( w, end = " ")
print("\n")
print( "The best tag sequence is:", end = " ")
for t in best_tagsequence: print (t, end = " ")
print("\n")
print( "The probability of the best tag sequence is:", prob_tagsequence)

Word 'flies' current best two-tag sequence: NN NN
Word 'are' current best two-tag sequence: NN BE
Word 'nice' current best two-tag sequence: BE JJ
The sentence was: fruit flies are nice 

The best tag sequence is: START NN NN BE JJ END 

The probability of the best tag sequence is: 7.946473148228979e-21
